In [1]:
# pip install pyfaidx biopython

In [1]:
import pandas as pd
import numpy as np
import torch  # Chuyển từ TF sang Torch
import os
from sklearn.metrics import confusion_matrix
import json
import time
from tqdm import tqdm
from metrics import compute_metrics
from data_preparation import prepare_csv_datasets

# Đường dẫn tới file model SPLAM (thường là splam_script.pt)
MODEL_PATH = r"D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SPLAM\pretrained_model\splam_script.pt"
PREPARED_FOLDER = "prepared_data/"
RESULTS_DIR = "results/"
BATCH_SIZE = 256  
CONTEXT = 400     
TARGET_LEN = 800
test_files = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv', 'test_data.csv']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data preparation

In [3]:
os.makedirs(PREPARED_FOLDER, exist_ok=True)
prepare_csv_datasets(test_files)

[23:12:22] Loading Genome...
🚀 Processing: test_1_1_1.csv (26310 rows)


100%|██████████| 26310/26310 [00:00<00:00, 35469.60it/s]



Type       | Sequence around Center (397-403) | Status
----------------------------------------------------------------------
Donor      | AG[GT]AA                       | ✅
Donor      | AG[GT]TT                       | ✅
Donor      | TT[GT]GA                       | ✅
Donor      | CA[GT]TT                       | ✅
Donor      | AG[GT]GC                       | ✅
Acceptor   | TC[AG]AA                       | ✅
Acceptor   | CC[AG]AA                       | ✅
Acceptor   | GG[AG]AC                       | ✅
Acceptor   | GC[AG]CT                       | ✅
Acceptor   | CC[AG]GC                       | ✅
✅ Saved to prepared_data/test_1_1_1.csv | Speed: 23310.31 seq/s

🚀 Processing: test_2_1_1.csv (35132 rows)


100%|██████████| 35132/35132 [00:00<00:00, 35367.36it/s]



Type       | Sequence around Center (397-403) | Status
----------------------------------------------------------------------
Donor      | AG[GT]CC                       | ✅
Donor      | TG[GT]GG                       | ✅
Donor      | TG[GT]AA                       | ✅
Donor      | AG[GT]TG                       | ✅
Donor      | TG[GC]AT                       | ❌
Acceptor   | CC[AG]GA                       | ✅
Acceptor   | GC[AG]TT                       | ✅
Acceptor   | AC[AG]AC                       | ✅
Acceptor   | GC[AG]CT                       | ✅
Acceptor   | TC[AG]GT                       | ✅
✅ Saved to prepared_data/test_2_1_1.csv | Speed: 22254.86 seq/s

🚀 Processing: test_4_1_1.csv (52776 rows)


...


Type       | Sequence around Center (397-403) | Status
----------------------------------------------------------------------
Donor      | AG[GT]GA                       | ✅
Donor      | CT[GT]GG                       | ✅
Donor      | TG[GT]AT                       | ✅
Donor      | AG[GT]GA                       | ✅
Donor      | AG[GT]AA                       | ✅
Acceptor   | CC[AG]TT                       | ✅
Acceptor   | GC[AG]CA                       | ✅
Acceptor   | GC[AG]AG                       | ✅
Acceptor   | AC[AG]GT                       | ✅
Acceptor   | CC[AG]GA                       | ✅
✅ Saved to prepared_data/test_4_1_1.csv | Speed: 23075.25 seq/s

🚀 Processing: test_10_1_1.csv (105708 rows)


100%|██████████| 105708/105708 [00:02<00:00, 36246.63it/s]



Type       | Sequence around Center (397-403) | Status
----------------------------------------------------------------------
Donor      | AG[GT]AG                       | ✅
Donor      | GA[GT]GA                       | ✅
Donor      | TG[GT]AA                       | ✅
Donor      | GA[GT]GT                       | ✅
Donor      | AG[GT]AA                       | ✅
Acceptor   | TG[AG]AT                       | ✅
Acceptor   | CC[AG]CT                       | ✅
Acceptor   | TC[AG]GT                       | ✅
Acceptor   | AC[AG]AA                       | ✅
Acceptor   | CC[AG]GC                       | ✅
✅ Saved to prepared_data/test_10_1_1.csv | Speed: 23105.87 seq/s

🚀 Processing: test_data.csv (938297 rows)


100%|██████████| 938297/938297 [00:25<00:00, 36711.36it/s]



Type       | Sequence around Center (397-403) | Status
----------------------------------------------------------------------
Donor      | AG[GT]AG                       | ✅
Donor      | AG[GT]AG                       | ✅
Donor      | AA[GT]GA                       | ✅
Donor      | AG[GT]TT                       | ✅
Donor      | CC[GT]GC                       | ✅
Acceptor   | TC[AG]TT                       | ✅
Acceptor   | AC[AG]CA                       | ✅
Acceptor   | GC[AG]TT                       | ✅
Acceptor   | GC[AG]CG                       | ✅
Acceptor   | TC[AG]GC                       | ✅
✅ Saved to prepared_data/test_data.csv | Speed: 23415.27 seq/s



In [4]:
df = pd.read_csv(r'D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SPLAM\prepared_data\test_1_1_1.csv')
# Kiểm tra 5 mẫu Donor đầu tiên
donor_samples = df[df['Splicing_types'] == 1]['sequence'].values[:5]
for i, seq in enumerate(donor_samples):
    # Lấy vùng trung tâm (index 5000 là nucleotide tại vị trí pos)
    center = seq[400:402] 
    print(f"Donor {i}: Vị trí trung tâm là '{center}' (Kỳ vọng thường là GT)")

# Kiểm tra 5 mẫu Acceptor đầu tiên
acceptor_samples = df[df['Splicing_types'] == 2]['sequence'].values[:5]
for i, seq in enumerate(acceptor_samples):
    center = seq[398:400]
    print(f"Acceptor {i}: Vị trí trung tâm là '{center}' (Kỳ vọng thường là AG)")

def diagnose_splice_sites(df, sample_size=5):
    print(f"{'Type':<10} | {'Window (-5 to +5 around center)':<20} | {'Found?'}")
    print("-" * 50)
    
    for label, name in [(1, 'Donor'), (2, 'Acceptor')]:
        samples = df[df['Splicing_types'] == label].sample(min(sample_size, len(df)))
        for _, row in samples.iterrows():
            seq = row['sequence']
            # Lấy 11 nucleotide (vị trí 5000 là trung tâm)
            window = seq[395:406]
            
            target = "GT" if label == 1 else "AG"
            found = "✅" if target in window else "❌"
            
            # Highlight vị trí trung tâm bằng dấu ngoặc []
            display_win = window[:5] + "[" + window[5:7] + "]" + window[7:]
            print(f"{name:<10} | {display_win:<20} | {found} (Target: {target})")

# Chạy sau khi load df từ prepared_data
diagnose_splice_sites(df)

Donor 0: Vị trí trung tâm là 'GT' (Kỳ vọng thường là GT)
Donor 1: Vị trí trung tâm là 'GT' (Kỳ vọng thường là GT)
Donor 2: Vị trí trung tâm là 'GT' (Kỳ vọng thường là GT)
Donor 3: Vị trí trung tâm là 'GT' (Kỳ vọng thường là GT)
Donor 4: Vị trí trung tâm là 'GT' (Kỳ vọng thường là GT)
Acceptor 0: Vị trí trung tâm là 'AG' (Kỳ vọng thường là AG)
Acceptor 1: Vị trí trung tâm là 'AG' (Kỳ vọng thường là AG)
Acceptor 2: Vị trí trung tâm là 'AG' (Kỳ vọng thường là AG)
Acceptor 3: Vị trí trung tâm là 'AG' (Kỳ vọng thường là AG)
Acceptor 4: Vị trí trung tâm là 'AG' (Kỳ vọng thường là AG)
Type       | Window (-5 to +5 around center) | Found?
--------------------------------------------------
Donor      | GTAAG[GT]GTGT        | ✅ (Target: GT)
Donor      | TAAAG[GT]GACA        | ✅ (Target: GT)
Donor      | GGAAG[GT]AAAG        | ✅ (Target: GT)
Donor      | TCCAG[GT]GGGG        | ✅ (Target: GT)
Donor      | AAGGT[GT]AAAG        | ✅ (Target: GT)
Acceptor   | TCCAG[GC]ATGC        | ✅ (Target: AG)
Acce

### Inference

In [2]:
# Load model SPLAM (TorchScript)
print(f"Loading SPLAM Model on {device}...")
model = torch.jit.load(MODEL_PATH)
model.to(device)
model.eval()
print("Successfully loaded SPLAM model.")

Loading SPLAM Model on cuda...
Successfully loaded SPLAM model.


In [ ]:
os.makedirs(RESULTS_DIR, exist_ok=True)

def one_hot_encode_splam(seq):
    """
    Mã hóa chuỗi DNA thành ma trận (4, 800)
    Thứ tự các hàng: 0:A, 1:C, 2:G, 3:T
    """
    seq = seq.upper()
    seq_code = np.zeros((4, len(seq)), dtype=np.float32)
    
    # Sử dụng numpy vectorization để tăng tốc
    sn = np.frombuffer(seq.encode('ascii'), dtype=np.int8)
    seq_code[0, sn == ord('A')] = 1.0
    seq_code[1, sn == ord('C')] = 1.0
    seq_code[2, sn == ord('G')] = 1.0
    seq_code[3, sn == ord('T')] = 1.0
    
    # Mọi ký tự khác (N, R, Y...) sẽ để là vector [0, 0, 0, 0]
    return seq_code

def debug_one_hot_splam(sequences, sample_size=1):
    """
    Kiểm tra xem dữ liệu mã hóa có khớp với chuỗi gốc và đúng shape SPLAM không
    """
    print("\n--- [CHECKING ONE-HOT ENCODING FOR SPLAM] ---")
    indices = np.random.choice(len(sequences), sample_size)
    map_dict = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}

    for idx in indices:
        original_seq = sequences[idx]
        encoded = one_hot_encode_splam(original_seq)

        # 1. Kiểm tra Shape (Bắt buộc phải là 4 x 800)
        if encoded.shape != (4, 800):
            print(f"❌ LỖI SHAPE: Nhận {encoded.shape}, kỳ vọng (4, 800)")
            continue

        # 2. Giải mã ngược (Reconstruct)
        # Lấy chỉ số của hàng có giá trị 1.0 tại mỗi cột
        decoded_list = []
        for col_idx in range(encoded.shape[1]):
            column = encoded[:, col_idx]
            if np.sum(column) == 0:
                decoded_list.append('N')
            else:
                decoded_list.append(map_dict[np.argmax(column)])
        
        decoded_seq = "".join(decoded_list)

        # 3. So sánh
        if original_seq.upper() == decoded_seq:
            print(f"✅ Mẫu {idx}: Khớp hoàn toàn!")
            # Kiểm tra motif trung tâm (Donor: GT, Acceptor: AG)
            print(f"   Motif trung tâm (398-402): {decoded_seq[398:402]}")
        else:
            print(f"⚠️ Mẫu {idx}: CÓ LỖI SAI LỆCH!")
            print(f"   Gốc: {original_seq[395:405]}")
            print(f"   Mã : {decoded_seq[395:405]}")

def get_batches(sequences, batch_size):
    for i in range(0, len(sequences), batch_size):
        batch_seqs = sequences[i:i + batch_size]
        # Encode và stack thành tensor (Batch, 4, 800)
        batch_data = np.array([one_hot_encode_splam(s) for s in batch_seqs])
        yield torch.tensor(batch_data, dtype=torch.float32).to(device)

pbar_files = tqdm(test_files, desc="Overall Progress")

for csv_file in pbar_files:
    path = os.path.join(PREPARED_FOLDER, csv_file)
    if not os.path.exists(path): continue
    
    df = pd.read_csv(path)
    debug_one_hot_splam(df['sequence'].values)
    y_true = df['Splicing_types'].values
    sequences = df['sequence'].values
    
    all_probs = []
    inference_start = time.time()
    
    with torch.no_grad():
        for batch_tensor in tqdm(get_batches(sequences, BATCH_SIZE), 
                                total=int(np.ceil(len(sequences)/BATCH_SIZE)), 
                                desc=" ↪ Inferencing", leave=False):
            
            # SPLAM Forward Pass
            # Output shape thường là (Batch, 3, 800) hoặc (Batch, 3) tùy version
            outputs = model(batch_tensor)
            
            # SPLAM thường dự đoán cho mọi vị trí hoặc vị trí trung tâm
            # Nếu output là (Batch, 3, 800), ta lấy tại index 400 (vị trí junction)
            if outputs.dim() == 3:
                # Lấy xác suất tại điểm giữa
                # SPLAM mapping: 0: None, 1: Acceptor, 2: Donor
                probs = outputs[:, :, 400].cpu().numpy()
            else:
                probs = outputs.cpu().numpy()
                
            all_probs.append(probs)

    avg_probs = np.vstack(all_probs)
    total_duration = time.time() - inference_start
    
    # --- MAPPING NHÃN THEO YÊU CẦU ---
    # SPLAM Gốc: 0=Null, 1=Acceptor, 2=Donor
    # Yêu cầu của bạn: 0=Null, 1=Donor, 2=Acceptor
    
    p_null = avg_probs[:, 0]
    p_acceptor_raw = avg_probs[:, 1] 
    p_donor_raw = avg_probs[:, 2]
    
    # Tạo mảng xác suất đã map: [p_null, p_donor, p_acceptor]
    probs_mapped = np.stack([p_null, p_donor_raw, p_acceptor_raw], axis=1)
    
    # Quyết định nhãn (Argmax hoặc Threshold)
    # preds_mapped = np.argmax(probs_mapped, axis=1)
    
    # (Tùy chọn) Áp dụng Threshold nếu cần độ nhạy cao
    THRESHOLD = 0.1
    preds_mapped = np.zeros(len(probs_mapped), dtype=int)
    for i in range(len(probs_mapped)):
        if p_donor_raw[i] > THRESHOLD and p_donor_raw[i] > p_acceptor_raw[i]:
            preds_mapped[i] = 1
        elif p_acceptor_raw[i] > THRESHOLD:
            preds_mapped[i] = 2
        else:
            preds_mapped[i] = 0

    # --- LƯU KẾT QUẢ ---
    try:
        # Giả sử hàm compute_metrics của bạn nhận (y_true, y_pred, probs)
        # kết quả trả về một dict
        from metrics import compute_metrics 
        results = compute_metrics(y_true, preds_mapped, probs=probs_mapped)
        
        results['benchmarking'] = {
            'total_inference_time_sec': round(total_duration, 4),
            'samples_per_second': round(len(df) / total_duration, 2)
        }
        
        cm = confusion_matrix(y_true, preds_mapped)
        output = {"metrics": results, "confusion_matrix": cm.tolist()}
        
        json_path = os.path.join(RESULTS_DIR, csv_file.replace('.csv', '_results.json'))
        with open(json_path, 'w') as f:
            json.dump(output, f, indent=4)
            
        print(f"✅ {csv_file} processed. Speed: {results['benchmarking']['samples_per_second']} seq/s")
            
    except Exception as e:
        print(f"\n❌ Error with {csv_file}: {e}")

print("\n✨ SPLAM INFERENCE COMPLETED!")

Overall Progress:   0%|          | 0/5 [00:00<?, ?it/s]


--- [CHECKING ONE-HOT ENCODING FOR SPLAM] ---
✅ Mẫu 19529: Khớp hoàn toàn!
   Motif trung tâm (398-402): AAGG


Overall Progress:  20%|██        | 1/5 [00:15<01:03, 15.82s/it]

✅ test_1_1_1.csv processed. Speed: 1682.13 seq/s

--- [CHECKING ONE-HOT ENCODING FOR SPLAM] ---
✅ Mẫu 2157: Khớp hoàn toàn!
   Motif trung tâm (398-402): TTGT


Overall Progress:  20%|██        | 1/5 [00:34<02:19, 34.89s/it]


KeyboardInterrupt: 